# Merge datasets

This script is mean to merge the ad_hominems dataset with the general dataset.

In [8]:
import numpy as np
import pandas as pd

general = pd.read_csv("../../data/general/ad_hominem_attacks.csv", sep=";")
adHominem = pd.read_csv("../../data/ad_hominem/reddit_ad_hominem.csv")

### 1. Preprocessing
The general dataset was analysed by us manually. We wrote 1 or 0 in the column with our name depending if we considered that row was a 'good' non-fallacy or a 'good' ad-hominem attack. If 2 out of 3 people put a 1 in a column, we are keeping it for our dataset.

In [9]:
general = general[ general["Pieter"] + general["Murilo"] + general["Eric"] >= 2]
general["isAdHominem"] = np.where(general["fallacies.df.Intended.Fallacy"] == "Ad Hominem",True, False)

In [10]:
general.head()

fallacies.df.Topic  \
0   Are humans to blame for certain animal extinct...   
1   Are humans to blame for certain animal extinct...   
7   Are humans to blame for certain animal extinct...   
9   Are humans to blame for certain animal extinct...   
11         Are Quentin Tarantinos movies too violent?   

   fallacies.df.Intended.Fallacy  \
0                     No Fallacy   
1                     No Fallacy   
7                     No Fallacy   
9                     Ad Hominem   
11                    Ad Hominem   

                                    fallacies.df.Text  Eric  Pieter  Murilo  \
0   Yes, human beings have hunted and eaten animal...   1.0       1       1   
1   Humans are not to be blamed for animal extinct...   0.0       1       1   
7         Humans don't care enough for living beings.   1.0       1       1   
9   Of course. You throw your garbage into the oce...   1.0       1       1   
11  Oh now, I'm not going to debate with you... Ha...   1.0       1       1   

    isAdHominem  
0         False  
1         False  
7         False  
9          True  
11         True

In [11]:
adHominem.head()

Unnamed: 0 archived     author_name  \
0          0     True    deckerparkes   
1          1     True       strapt313   
2          2     True  ZeusThunder369   
3          3     True  TheToastIsBlue   
4          4     True             NaN   

                                                body  \
0  What makes corporations different in this case...   
1  >there are arguments that they will consider b...   
2  Basically to believe a patriarchy exists, you ...   
3  The punishment for heresy was being burned at ...   
4  No it doesn't. Sex is defined by DNA. DNA cann...   

                                           body_html controversiality  \
0  <div class="md"><p>What makes corporations dif...                0   
1  <div class="md"><blockquote>\n<p>there are arg...                0   
2  <!-- SC_OFF --><div class="md"><p>Basically to...                0   
3  <div class="md"><p>The punishment for heresy w...                0   
4  <div class="md"><p>No it doesn&#39;t. Sex is d...                0   

        created   created_utc  delta downs    ...           id    link_id  \
0  1.466105e+09  1466076301.0  False     0    ...      d4bfrtv  t3_4ocqwc   
1  1.424406e+09  1424377113.0  False     0    ...      coqorty  t3_2wbn3s   
2  1.486987e+09  1486958071.0  False     0    ...       5tqqra        NaN   
3  1.490136e+09  1490107248.0  False     0    ...      df7ulkr  t3_60mna9   
4  1.444970e+09  1444940749.0  False     0    ...      cw11y9d  t3_3ov1gq   

  mod_reports        name   parent_id  \
0          []  t1_d4bfrtv  t1_d4bfkyw   
1          []  t1_coqorty  t1_coqer7x   
2          []   t3_5tqqra         NaN   
3          []  t1_df7ulkr  t1_df7topk   
4          []  t1_cw11y9d  t1_cw11qpe   

                                               title ups user_reports  \
0                                                NaN   9           []   
1                                                NaN   1           []   
2  CMV: I don't know how one can believe a patria...   5           []   
3                                                NaN  42           []   
4                                                NaN  -9           []   

  violated_rule ad_hominem  
0           0.0      False  
1           2.0       True  
2           0.0      False  
3           0.0      False  
4           0.0      False  

[5 rows x 22 columns]

In [30]:
df_ad_hominems = pd.concat([adHominem['body'], adHominem['ad_hominem']], axis=1, keys=['body', 'isAdHominem'])
df_general = pd.concat([general['fallacies.df.Text'], general['isAdHominem']], axis=1, keys=['body', 'isAdHominem'])
df_merged = df_ad_hominems.append(df_general)

df_merged["body"] = df_merged["body"].astype(str)
df_merged["isAdHominem"] = df_merged["isAdHominem"].astype(bool)
df_merged = df_merged[~df_merged.isin([np.nan, np.inf, -np.inf, 'nan']).any(1)] ## Remove rows with NaN values

print(df_ad_hominems.shape)
print(df_general.shape)
print(df_merged.shape)

(29281, 2)
(382, 2)
(29660, 2)


### 2. Export

After running the cell below, the merged_datasets file is created and includes both datasets merged. Keep in mind that the first 29281 rows belong originally to the ad_hominems dataset and the following 382 rows belong to the general dataset. The first column just includes the id of every row from each dataset. So it starts with id 0 (first row of ad_hominems dataset), then follows to id 29280 (last row of ad_hominems dataset), then starts over with id 0 again (first row of general dataset) and ends with id 588 (last row of general dataset).

In [31]:
df_merged.to_csv("../../data/merged_datasets.csv")